In [1]:
import copy
import math
import numpy as np
import scipy.linalg as la
from qiskit import *
import qiskit
from qiskit.tools.qi.qi import random_unitary_matrix
from qiskit.mapper import two_qubit_kak
from qiskit.mapper import CouplingMap, Layout
from qiskit.extensions.standard import SwapGate
from qiskit.converters import circuit_to_dag
%load_ext Cython

In [2]:
IBMQ.load_accounts()
backend = IBMQ.get_backend('ibmq_poughkeepsie')
basis = backend.configuration().to_dict()['basis_gates']
coupling_map = CouplingMap(backend.configuration().to_dict()['coupling_map'])

In [20]:
%%cython --cplus --compile-args=-mmacosx-version-min=10.9
cimport cython
import numpy as np
from libc.stdlib cimport calloc, free
from libcpp.vector cimport vector
from libcpp.set cimport set as cset

from qiskit.mapper._layout import Layout

cdef extern from "<random>" namespace "std":
    cdef cppclass mt19937:
        mt19937() # 32-bit Mersenne Twister 19937 generator
        mt19937(unsigned int seed)

    cdef cppclass normal_distribution[T]:
        normal_distribution()
        normal_distribution(T a, T b)
        T operator()(mt19937 rnd_gen)


cdef class SHIFTED_NORMAL_RNG:
    cdef:
        mt19937 rnd_gen
        normal_distribution[double] distro
    # Some Python overhead here, but init once per call to
    # _layer_permutation
    def __cinit__(self, unsigned int seed, unsigned int num_qubits):
        self.rnd_gen = mt19937(seed)
        self.distro = normal_distribution[double](0.0,1.0/num_qubits)

    cpdef double rand(self):
        return 1.0+self.distro(self.rnd_gen)


cdef class EdgeCollection:
    cdef vector[unsigned int] _edges

    cpdef void add(self, unsigned int edge_start,
                   unsigned int edge_end):
        self._edges.push_back(edge_start)
        self._edges.push_back(edge_end)
    
    @property
    def size(self):
        return self._edges.size()
    @cython.boundscheck(False)
    def edges(self):
        cdef size_t kk
        out = np.zeros(self._edges.size(), dtype=np.uint32)
        for kk in range(self._edges.size()):
            out[kk] = self._edges[kk]
        return out

cdef class NLayout:
    cdef:
        unsigned int l2p_len
        unsigned int p2l_len
        unsigned int * logic_to_phys
        unsigned int * phys_to_logic
        
    def __cinit__(self, unsigned int num_logical,
                  unsigned int num_physical):
        
        self.l2p_len = num_logical
        self.p2l_len = num_physical
        self.logic_to_phys = <unsigned int *>calloc(num_logical,sizeof(unsigned int))
        self.phys_to_logic = <unsigned int *>calloc(num_physical, sizeof(unsigned int))
    
    def __dealloc__(self):
        if self.logic_to_phys is not NULL:
            free(self.logic_to_phys)
            self.logic_to_phys = NULL
        if self.phys_to_logic is not NULL:
            free(self.phys_to_logic)
            self.phys_to_logic = NULL
            
    @property
    def logic_to_phys(self):
        cdef size_t kk
        out = np.zeros(self.l2p_len, dtype=np.int32)
        for kk in range(self.l2p_len):
            out[kk] = self.logic_to_phys[kk]
        return out
    
    @property
    def phys_to_logic(self):
        cdef size_t kk
        out = np.zeros(self.p2l_len, dtype=np.int32)
        for kk in range(self.p2l_len):
            out[kk] = self.phys_to_logic[kk]
        return out
    
    @cython.boundscheck(False)
    cdef NLayout copy(self):
        cdef NLayout out = NLayout(self.l2p_len, self.p2l_len)
        cdef size_t kk
        for kk in range(self.l2p_len):
            out.logic_to_phys[kk] = self.logic_to_phys[kk]
        for kk in range(self.p2l_len):
            out.phys_to_logic[kk] = self.phys_to_logic[kk]
        return out
            
    @cython.boundscheck(False)
    cdef void swap(self, unsigned int idx1, unsigned int idx2):
        cdef unsigned int temp1, temp2
        temp1 = self.phys_to_logic[idx1]
        temp2 = self.phys_to_logic[idx2]
        self.phys_to_logic[idx1] = temp2
        self.phys_to_logic[idx2] = temp1
        self.logic_to_phys[self.phys_to_logic[idx1]] = idx1
        self.logic_to_phys[self.phys_to_logic[idx2]] = idx2
        
    
    cpdef object to_layout(self, object qregs):
        out = Layout()
        cdef unsigned int main_idx = 0
        cdef size_t idx
        for qreg in qregs.values():
            for idx in range(qreg.size):
                out[(qreg, idx)] = self.logic_to_phys[main_idx]
                main_idx += 1
        return out
    
    
cpdef NLayout nlayout_from_layout(object layout,
                          object qregs, 
                          unsigned int physical_qubits):
    cdef size_t ind
    cdef list sizes = [qr.size for qr in qregs.values()]
    cdef int[::1] reg_idx = np.cumsum([0]+sizes, dtype=np.int32)
    cdef unsigned int logical_qubits = sum(sizes)

    cdef dict regint = {}
    for ind, qreg in enumerate(qregs.values()):
        regint[qreg] = ind

    cdef NLayout out = NLayout(logical_qubits, physical_qubits)
    cdef object key, val
    cdef dict merged_dict = {**layout._p2v, **layout._v2p}
    for key, val in merged_dict.items():
        if isinstance(key, tuple):
            out.logic_to_phys[reg_idx[regint[key[0]]]+key[1]] = val
        else:
            out.phys_to_logic[key] = reg_idx[regint[val[0]]]+val[1]
    return out

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double compute_cost(double[:, ::1] scale, unsigned int * logic_to_phys,
                          int[::1] gates, unsigned int num_gates) nogil:
    cdef unsigned int ii, jj, kk
    cdef double cost = 0.0
    for kk in range(num_gates):
        ii = logic_to_phys[gates[2*kk]]
        jj = logic_to_phys[gates[2*kk+1]]
        cost += scale[ii,jj]
    return cost

@cython.nonecheck(False)
@cython.boundscheck(False)
@cython.wraparound(False)
cdef compute_random_scaling(double[:, ::1] scale, double[:, ::1] cdist2, double * rand, int num_qubits):
    cdef size_t ii, jj, idx=0
    for ii in range(num_qubits):
        for jj in range(ii):
            scale[ii,jj] = rand[idx]*cdist2[ii,jj]
            scale[jj,ii] = scale[ii,jj]
            idx += 1


@cython.nonecheck(False)
@cython.boundscheck(False)
@cython.wraparound(False)
def swap_trial(int num_qubits, NLayout int_layout, int[::1] int_qubit_subset,
               int[::1] gates, double[:, ::1] cdist2, double[:, ::1] cdist, 
               int[::1] edges, double[:, ::1] scale, SHIFTED_NORMAL_RNG rng):
    
    cdef EdgeCollection opt_edges = EdgeCollection()
    cdef NLayout optimal_layout, new_layout, trial_layout = int_layout.copy()
    
    cdef unsigned int num_gates = gates.shape[0]//2
    cdef unsigned int num_edges = edges.shape[0]//2
    
    cdef unsigned int need_copy, cost_reduced
    cdef unsigned int depth_step = 1
    cdef unsigned int depth_max = 2 * num_qubits + 1
    cdef double min_cost, new_cost, dist
    
    cdef unsigned int start_edge, end_edge, start_qubit, end_qubit
    cdef unsigned int optimal_start, optimal_end, optimal_start_qubit, optimal_end_qubit
    
    cdef size_t idx
    
    # Compute randomized distance
    cdef double * rand = <double *>calloc(num_qubits*(num_qubits+1)//2, sizeof(double))
    for idx in range(num_qubits*(num_qubits+1)//2):
        rand[idx] = rng.rand()
    compute_random_scaling(scale, cdist2, rand, num_qubits)
    free(rand)
    rand = NULL
    
    # Convert int qubit array to c++ set
    cdef cset[unsigned int] qubit_set
    cdef cset[unsigned int] input_qubit_set
    
    for idx in range(int_qubit_subset.shape[0]):
        input_qubit_set.insert(int_qubit_subset[idx])
    
    # Loop over depths from 1 up to a maximum depth
    while depth_step < depth_max:
        qubit_set = input_qubit_set
        # While there are still qubits available
        while not qubit_set.empty():
            # Compute the objective function
            min_cost = compute_cost(scale, trial_layout.logic_to_phys,
                                   gates, num_gates)
            # Try to decrease objective function
            cost_reduced = 0

            # Loop over edges of coupling graph
            need_copy = 1
            for idx in range(num_edges):
                start_edge = edges[2*idx]
                end_edge = edges[2*idx+1]
                start_qubit = trial_layout.phys_to_logic[start_edge]
                end_qubit =  trial_layout.phys_to_logic[end_edge]
                # Are the qubits available?
                if  qubit_set.count(start_qubit) and qubit_set.count(end_qubit):
                    # Try this edge to reduce the cost
                    if need_copy:
                        new_layout = trial_layout.copy()
                        need_copy = 0
                    new_layout.swap(start_edge, end_edge)
                    # Compute the objective function
                    new_cost = compute_cost(scale, new_layout.logic_to_phys,
                                   gates, num_gates)
                    # Record progress if we succceed
                    if new_cost < min_cost:
                        cost_reduced = True
                        min_cost = new_cost
                        optimal_layout = new_layout
                        optimal_start = start_edge
                        optimal_end = end_edge
                        optimal_start_qubit = start_qubit
                        optimal_end_qubit = end_qubit
                        need_copy = 1
                    else:
                        new_layout.swap(start_edge, end_edge)

            # After going over all edges
            # Were there any good swap choices?
            if cost_reduced:
                qubit_set.erase(optimal_start_qubit)
                qubit_set.erase(optimal_end_qubit)
                trial_layout = optimal_layout
                opt_edges.add(optimal_start, optimal_end)
            else:
                break

        # We have either run out of swap pairs to try or
        # failed to improve the cost.

        # Compute the coupling graph distance
        dist = compute_cost(cdist, trial_layout.logic_to_phys,
                                   gates, num_gates)
        # If all gates can be applied now, we are finished.
        # Otherwise we need to consider a deeper swap circuit
        if dist == num_gates:
            break

        # Increment the depth
        depth_step += 1

    # Either we have succeeded at some depth d < dmax or failed
    dist = compute_cost(cdist, trial_layout.logic_to_phys,
                                   gates, num_gates)
    
    return dist, opt_edges, trial_layout, depth_step


In [21]:
def my_layer_permutation(layer_partition, initial_layout, layout, qubit_subset,
                           coupling, trials, qregs, rng):

        logger.debug("layer_permutation: layer_partition = %s",
                     pformat(layer_partition))
        logger.debug("layer_permutation: layout = %s",
                     pformat(layout.get_virtual_bits()))
        logger.debug("layer_permutation: qubit_subset = %s",
                     pformat(qubit_subset))
        logger.debug("layer_permutation: trials = %s", trials)

        gates = []  # list of lists of tuples [[(register, index), ...], ...]
        for gate_args in layer_partition:
            if len(gate_args) > 2:
                raise TranspilerError("Layer contains > 2-qubit gates")
            elif len(gate_args) == 2:
                gates.append(tuple(gate_args))
        logger.debug("layer_permutation: gates = %s", pformat(gates))

        # Can we already apply the gates? If so, there is no work to do.
        dist = sum([coupling.distance(layout[g[0]], layout[g[1]])
                    for g in gates])
        logger.debug("layer_permutation: distance = %s", dist)
        if dist == len(gates):
            logger.debug("layer_permutation: nothing to do")
            circ = DAGCircuit()
            for register in layout.get_virtual_bits().keys():
                if register[0] not in circ.qregs.values():
                    circ.add_qreg(register[0])
            #circ.add_basis_element("swap", 2)
            return True, circ, 0, layout, (not bool(gates))

        # Begin loop over trials of randomized algorithm
        num_qubits = len(layout)
        best_depth = inf  # initialize best depth
        best_edges = None # best edges found
        best_circuit = None  # initialize best swap circuit
        best_layout = None  # initialize best final layout

        cdist2 = coupling._dist_matrix**2
        # Scaling matrix
        scale = np.zeros((num_qubits, num_qubits))

        int_qubit_subset = regtuple_to_numeric(qubit_subset, qregs)
        int_gates = gates_to_idx(gates, qregs)
        int_layout = nlayout_from_layout(layout, qregs, coupling.size())
        numeric_init_layout = nlayout_from_layout(initial_layout, qregs, coupling.size())
        
        trial_circuit = DAGCircuit()  # SWAP circuit for this trial
        for register in layout.get_virtual_bits().keys():
            if register[0] not in trial_circuit.qregs.values():
                trial_circuit.add_qreg(register[0])
        
        slice_circuit = DAGCircuit()  # circuit for this swap slice
        for register in layout.get_virtual_bits().keys():
            if register[0] not in slice_circuit.qregs.values():
                slice_circuit.add_qreg(register[0])
        #slice_circuit.add_basis_element("swap", 2)
        edges = np.asarray(coupling.get_edges(),dtype=np.int32).ravel()
        cdist = coupling._dist_matrix
        for trial in range(trials):
            logger.debug("layer_permutation: trial %s", trial)
            # This is one Trial --------------------------------------
            dist, optim_edges, trial_layout, depth_step = swap_trial(num_qubits, int_layout, 
                                                                     int_qubit_subset, 
                                                                     int_gates, cdist2,
                                                                     cdist, edges, scale,
                                                                     rng)
            
            
            logger.debug("layer_permutation: final distance for this trial = %s", dist)
            if dist == len(gates) and depth_step < best_depth:
                logger.debug("layer_permutation: got circuit with improved depth %s",
                             depth_step)
                best_edges = optim_edges
                best_layout = trial_layout
                best_depth = min(best_depth, depth_step)

            # Break out of trial loop if we found a depth 1 circuit
            # since we can't improve it further
            if best_depth == 1:
                break

        # If we have no best circuit for this layer, all of the
        # trials have failed
        if best_layout is None:
            logger.debug("layer_permutation: failed!")
            return False, None, None, None, False
        
        edgs = best_edges.edges()
        for kk in range(best_edges.size//2):
            slice_circuit.apply_operation_back(SwapGate(initial_layout[edgs[2*kk]],
                                                        initial_layout[edgs[2*kk+1]]))
        trial_circuit.extend_back(slice_circuit)
        best_circuit = trial_circuit

        # Otherwise, we return our result for this layer
        logger.debug("layer_permutation: success!")
        best_lay = best_layout.to_layout(qregs)
        return True, best_circuit, best_depth, best_lay, False

In [22]:
from logging import getLogger
from pprint import pformat
from math import inf
import numpy as np

from qiskit.transpiler._basepasses import TransformationPass
from qiskit.transpiler.exceptions import TranspilerError
from qiskit.dagcircuit import DAGCircuit
from qiskit.extensions.standard import SwapGate
from qiskit.mapper import Layout
from qiskit.transpiler.passes.mapping.barrier_before_final_measurements import BarrierBeforeFinalMeasurements

logger = getLogger(__name__)


# Notes:
# 1. Measurements may occur and be followed by swaps that result in repeated
# measurement of the same qubit. Near-term experiments cannot implement
# these circuits, so some care is required when using this mapper
# with experimental backend targets.
# 2. We do not use the fact that the input state is zero to simplify
# the circuit.


class MySwap(TransformationPass):
    """
    Maps a DAGCircuit onto a `coupling_map` adding swap gates.

    Uses a randomized algorithm.
    """

    def __init__(self, coupling_map, initial_layout=None,
                 trials=20, seed=None):
        """
        Map a DAGCircuit onto a `coupling_map` using swap gates.

        If initial_layout is not None, we assume the input circuit
        has been layed out before running this pass, and that
        the layout process yields a DAG, coupling map, and layout
        with the following properties:

        1. All three have the same number of qubits
        2. The layout a bijection from the DAG qubits to the coupling map

        For this mapping pass, it may also be necessary that

        3. The coupling map is a connected graph

        If these are not satisfied, the behavior is undefined.

        Args:
            coupling_map (CouplingMap): Directed graph representing a coupling
                map.
            initial_layout (Layout): initial layout of qubits in mapping
            trials (int): maximum number of iterations to attempt
            seed (int): seed for random number generator
        """
        super().__init__()
        self.coupling_map = coupling_map
        self.initial_layout = initial_layout
        self.input_layout = None
        self.trials = trials
        self.seed = seed
        self.rng = None
        self.requires.append(BarrierBeforeFinalMeasurements())
        self.qregs = None
        

    def run(self, dag):
        """
        Run the StochasticSwap pass on `dag`.

        Args:
            dag (DAGCircuit): DAG to map.

        Returns:
            DAGCircuit: A mapped DAG.

        Raises:
            TranspilerError: if the coupling map or the layout are not
            compatible with the DAG
        """
        
        if self.initial_layout is None:
            if self.property_set["layout"]:
                self.initial_layout = self.property_set["layout"]
            else:
                self.initial_layout = Layout.generate_trivial_layout(*dag.qregs.values())

        if len(dag.qubits()) != len(self.initial_layout):
            raise TranspilerError('The layout does not match the amount of qubits in the DAG')

        if len(self.coupling_map.physical_qubits) != len(self.initial_layout):
            raise TranspilerError(
                "Mappers require to have the layout to be the same size as the coupling map")

        self.input_layout = self.initial_layout.copy()
        self.qregs = dag.qregs
        
        if self.seed is None:
            self.seed = np.random.randint(1, np.iinfo(np.int32).max-1)
        self.rng = SHIFTED_NORMAL_RNG(self.seed, len(self.initial_layout))
        new_dag = self._mapper(dag, self.coupling_map, trials=self.trials)
        # self.property_set["layout"] = self.initial_layout
        return new_dag

    def _layer_permutation(self, layer_partition, layout, qubit_subset,
                           coupling, trials):
        
        return my_layer_permutation(layer_partition, self.initial_layout,
                                    layout, qubit_subset,
                                    coupling, trials, self.qregs, self.rng)

    def _layer_update(self, i, first_layer, best_layout, best_depth,
                      best_circuit, layer_list):
        """Provide a DAGCircuit for a new mapped layer.

        i (int) = layer number
        first_layer (bool) = True if this is the first layer in the
            circuit with any multi-qubit gates
        best_layout (Layout) = layout returned from _layer_permutation
        best_depth (int) = depth returned from _layer_permutation
        best_circuit (DAGCircuit) = swap circuit returned
            from _layer_permutation
        layer_list (list) = list of DAGCircuit objects for each layer,
            output of DAGCircuit layers() method

        Return a DAGCircuit object to append to the output DAGCircuit
        that the _mapper method is building.
        """
        layout = best_layout
        logger.debug("layer_update: layout = %s", pformat(layout))
        logger.debug("layer_update: self.initial_layout = %s", pformat(self.initial_layout))
        dagcircuit_output = DAGCircuit()
        for register in layout.get_virtual_bits().keys():
            if register[0] not in dagcircuit_output.qregs.values():
                dagcircuit_output.add_qreg(register[0])

        # If this is the first layer with multi-qubit gates,
        # output all layers up to this point and ignore any
        # swap gates. Set the initial layout.
        if first_layer:
            logger.debug("layer_update: first multi-qubit gate layer")
            # Output all layers up to this point
            for j in range(i + 1):
                # Make qubit edge map and extend by classical bits
                edge_map = layout.combine_into_edge_map(self.initial_layout)
                for bit in dagcircuit_output.clbits():
                    edge_map[bit] = bit
                dagcircuit_output.compose_back(layer_list[j]["graph"], edge_map)
        # Otherwise, we output the current layer and the associated swap gates.
        else:
            # Output any swaps
            if best_depth > 0:
                logger.debug("layer_update: there are swaps in this layer, "
                             "depth %d", best_depth)
                dagcircuit_output.extend_back(best_circuit)
            else:
                logger.debug("layer_update: there are no swaps in this layer")
            # Make qubit edge map and extend by classical bits
            edge_map = layout.combine_into_edge_map(self.initial_layout)
            for bit in dagcircuit_output.clbits():
                edge_map[bit] = bit
            # Output this layer
            dagcircuit_output.compose_back(layer_list[i]["graph"], edge_map)

        return dagcircuit_output

    def _mapper(self, circuit_graph, coupling_graph,
                trials=20):
        """Map a DAGCircuit onto a CouplingMap using swap gates.

        Use self.initial_layout for the initial layout.

        Args:
            circuit_graph (DAGCircuit): input DAG circuit
            coupling_graph (CouplingMap): coupling graph to map onto
            trials (int): number of trials.
            seed (int): initial seed.

        Returns:
            DAGCircuit: object containing a circuit equivalent to
                circuit_graph that respects couplings in coupling_graph
            Layout: a layout object mapping qubits of circuit_graph into
                qubits of coupling_graph. The layout may differ from the
                initial_layout if the first layer of gates cannot be
                executed on the initial_layout, since in this case
                it is more efficient to modify the layout instead of swapping
            Dict: a final-layer qubit permutation

        Raises:
            TranspilerError: if there was any error during the mapping
                or with the parameters.
        """
        # Schedule the input circuit by calling layers()
        layerlist = list(circuit_graph.layers())
        logger.debug("schedule:")
        for i, v in enumerate(layerlist):
            logger.debug("    %d: %s", i, v["partition"])

        if self.initial_layout is not None:
            qubit_subset = self.initial_layout.get_virtual_bits().keys()
        else:
            # Supply a default layout for this dag
            self.initial_layout = Layout()
            physical_qubit = 0
            for qreg in circuit_graph.qregs.values():
                for index in range(qreg.size):
                    self.initial_layout[(qreg, index)] = physical_qubit
                    physical_qubit += 1
            qubit_subset = self.initial_layout.get_virtual_bits().keys()
            # Restrict the coupling map to the image of the layout
            coupling_graph = coupling_graph.subgraph(
                self.initial_layout.get_physical_bits().keys())
            if coupling_graph.size() < len(self.initial_layout):
                raise TranspilerError("Coupling map too small for default layout")
            self.input_layout = self.initial_layout.copy()

        # Find swap circuit to preceed to each layer of input circuit
        layout = self.initial_layout.copy()

        # Construct an empty DAGCircuit with the same set of
        # qregs and cregs as the input circuit
        dagcircuit_output = DAGCircuit()
        dagcircuit_output.name = circuit_graph.name
        for qreg in circuit_graph.qregs.values():
            dagcircuit_output.add_qreg(qreg)
        for creg in circuit_graph.cregs.values():
            dagcircuit_output.add_creg(creg)

        # Make a trivial wire mapping between the subcircuits
        # returned by _layer_update and the circuit we build
        identity_wire_map = {}
        for qubit in circuit_graph.qubits():
            identity_wire_map[qubit] = qubit
        for bit in circuit_graph.clbits():
            identity_wire_map[bit] = bit

        first_layer = True  # True until first layer is output
        logger.debug("initial_layout = %s", layout)

        # Iterate over layers
        for i, layer in enumerate(layerlist):

            # Attempt to find a permutation for this layer
            success_flag, best_circuit, best_depth, best_layout, trivial_flag \
                = self._layer_permutation(layer["partition"], layout,
                                          qubit_subset, coupling_graph,
                                          trials)
            logger.debug("mapper: layer %d", i)
            logger.debug("mapper: success_flag=%s,best_depth=%s,trivial_flag=%s",
                         success_flag, str(best_depth), trivial_flag)

            # If this fails, try one gate at a time in this layer
            if not success_flag:
                logger.debug("mapper: failed, layer %d, "
                             "retrying sequentially", i)
                serial_layerlist = list(layer["graph"].serial_layers())

                # Go through each gate in the layer
                for j, serial_layer in enumerate(serial_layerlist):

                    success_flag, best_circuit, best_depth, best_layout, trivial_flag = \
                        self._layer_permutation(
                            serial_layer["partition"],
                            layout, qubit_subset,
                            coupling_graph,
                            trials)
                    logger.debug("mapper: layer %d, sublayer %d", i, j)
                    logger.debug("mapper: success_flag=%s,best_depth=%s,"
                                 "trivial_flag=%s",
                                 success_flag, str(best_depth), trivial_flag)

                    # Give up if we fail again
                    if not success_flag:
                        raise TranspilerError("mapper failed: " +
                                              "layer %d, sublayer %d" %
                                              (i, j) + ", \"%s\"" %
                                              serial_layer["graph"].qasm(
                                                  no_decls=True,
                                                  aliases=layout))

                    # If this layer is only single-qubit gates,
                    # and we have yet to see multi-qubit gates,
                    # continue to the next inner iteration
                    if trivial_flag and first_layer:
                        logger.debug("mapper: skip to next sublayer")
                        continue

                    if first_layer:
                        self.initial_layout = layout

                    # Update the record of qubit positions
                    # for each inner iteration
                    layout = best_layout
                    # Update the DAG
                    dagcircuit_output.extend_back(
                        self._layer_update(j,
                                           first_layer,
                                           best_layout,
                                           best_depth,
                                           best_circuit,
                                           serial_layerlist),
                        identity_wire_map)
                    if first_layer:
                        first_layer = False

            else:
                # Update the record of qubit positions for each iteration
                layout = best_layout

                if first_layer:
                    self.initial_layout = layout

                # Update the DAG
                dagcircuit_output.extend_back(
                    self._layer_update(i,
                                       first_layer,
                                       best_layout,
                                       best_depth,
                                       best_circuit,
                                       layerlist),
                    identity_wire_map)

                if first_layer:
                    first_layer = False

        # This is the final edgemap. We might use it to correctly replace
        # any measurements that needed to be removed earlier.
        logger.debug("mapper: self.initial_layout = %s", pformat(self.initial_layout))
        logger.debug("mapper: layout = %s", pformat(layout))
        last_edgemap = layout.combine_into_edge_map(self.initial_layout)
        logger.debug("mapper: last_edgemap = %s", pformat(last_edgemap))

        # If first_layer is still set, the circuit only has single-qubit gates
        # so we can use the initial layout to output the entire circuit
        # This code is dead due to changes to first_layer above.
        if first_layer:
            logger.debug("mapper: first_layer flag still set")
            layout = self.initial_layout
            for i, layer in enumerate(layerlist):
                edge_map = layout.combine_into_edge_map(self.initial_layout)
                dagcircuit_output.compose_back(layer["graph"], edge_map)

        return dagcircuit_output


def regtuple_to_numeric(items, qregs):
    """Takes (QuantumRegister, int) tuples and converts
    them into an integer array.

    Args:
        items (list): List of tuples of (QuantumRegister, int)
                      to convert.
        qregs (dict): List of )QuantumRegister, int) tuples.
    Returns:
        ndarray: Array of integers.

    """
    sizes = [qr.size for qr in qregs.values()]
    reg_idx = np.cumsum([0]+sizes)
    regint = {}
    for ind, qreg in enumerate(qregs.values()):
        regint[qreg] = ind
    out = np.zeros(len(items), dtype=np.int32)
    for idx, val in enumerate(items):
        out[idx] = reg_idx[regint[val[0]]]+val[1]
    return out


def gates_to_idx(gates, qregs):
    """Converts gate tuples into a nested list of integers.

    Args:
        gates (list): List of (QuantumRegister, int) pairs
                      representing gates.
        qregs (dict): List of )QuantumRegister, int) tuples.

    Returns:
        list: Nested list of integers for gates.
    """
    sizes = [qr.size for qr in qregs.values()]
    reg_idx = np.cumsum([0]+sizes)
    regint = {}
    for ind, qreg in enumerate(qregs.values()):
        regint[qreg] = ind
    out = np.zeros(2*len(gates), dtype=np.int32)
    for idx, gate in enumerate(gates):
        out[2*idx] = reg_idx[regint[gate[0][0]]]+gate[0][1]
        out[2*idx+1] = reg_idx[regint[gate[1][0]]]+gate[1][1]
    return out

In [23]:
from qiskit.transpiler.passes import *
from qiskit.transpiler.passes.mapping import *


pm = qiskit.transpiler.PassManager()
pm.append(Unroller(basis))
pm.append(BarrierBeforeFinalMeasurements())
pm.append(DenseLayout(coupling_map))
pm.append(MySwap(coupling_map))
pm.append(Decompose(SwapGate))
pm.append(CXDirection(coupling_map))
pm.append(CXCancellation())
pm.append(Unroller(['u1', 'u2', 'u3', 'id', 'cx']))
pm.append(Optimize1qGates())

# QV

In [24]:
def build_qv_circuit(seed, n, depth):
    """Create a quantum program containing model circuits.

    The model circuits consist of layers of Haar random
    elements of SU(4) applied between corresponding pairs
    of qubits in a random bipartition.

    name = leading name of circuits
    n = number of qubits
    depth = ideal depth of each model circuit (over SU(4))

    """
    np.random.seed(seed)
    q = QuantumRegister(n, "q")
    c = ClassicalRegister(n, "c")
    # Create measurement subcircuit
    qc = QuantumCircuit(q,c)
    # For each layer
    for j in range(depth):
        # Generate uniformly random permutation Pj of [0...n-1]
        perm = np.random.permutation(n)
        # For each pair p in Pj, generate Haar random SU(4)
        # Decompose each SU(4) into CNOT + SU(2) and add to Ci
        for k in range(math.floor(n/2)):
            qubits = [int(perm[2*k]), int(perm[2*k+1])]
            U = random_unitary_matrix(4)
            for gate in two_qubit_kak(U):
                i0 = qubits[gate["args"][0]]
                if gate["name"] == "cx":
                    i1 = qubits[gate["args"][1]]
                    qc.cx(q[i0], q[i1])
                elif gate["name"] == "u1":
                    qc.u1(gate["params"][2], q[i0])
                elif gate["name"] == "u2":
                    qc.u2(gate["params"][1], gate["params"][2],
                                 q[i0])
                elif gate["name"] == "u3":
                    qc.u3(gate["params"][0], gate["params"][1],
                                 gate["params"][2], q[i0])
                elif gate["name"] == "id":
                    pass  # do nothing
    qc.measure(q,c)
    return qc

In [25]:
qc = build_qv_circuit(123456, 20, 5)

In [26]:
compile(qc, backend=backend, pass_manager=pm);